In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.optimize import minimize

In [2]:
def risk_parity_weight(cov_mat, risk_prop, equity_prop):    
    sigma = np.matrix(cov_mat.values)

    def objfun(x):
        tmp = (sigma * np.matrix(x).T).A1
        risk = x * tmp
        var = sum(risk)
        delta_risk = ((sum(risk[0:3]) - risk_prop[0]*var)**2 + 
                      (sum(risk[3:5]) - risk_prop[1]*var)**2 +
                      (sum(risk[5:9]) - risk_prop[2]*var)**2)
        return delta_risk

    x0 = np.ones(sigma.shape[0]) / sigma.shape[0]  
    bnds = tuple((0,None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1},
            {'type': 'ineq', 'fun': lambda x: equity_prop - sum(x[0:3])/sum(x)})
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}

    res = minimize(objfun, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
    wts = pd.Series(index=cov_mat.index, data=res['x'])
    return wts / wts.sum() * 1.0

In [3]:
def standardize(tmp):
    if isinstance(tmp, pd.Series):
        mu = tmp.mean()
        sigma = tmp.std()
        tmp = (tmp - mu)/sigma
    return tmp   

In [4]:
pool=pd.read_csv("pool.csv", encoding='utf-8')[:-1]
pool = pool.set_index("Date")
pool = pool.fillna(pool.mean())
for i in pool.columns:
    pool[i] = standardize(pool[i])
pool.head()

,iShares Russell 1000 Value ETF,Praxis Value Index A,Wells Fargo C&B Large Cap Value A,Commerce Value,Metropolitan West Total Return Bd I,iShares Core US Aggregate Bond ETF,Schwab S&P 500 Index,Vanguard Real Estate ETF,iShares Gold Trust
Date,,,,,,,,,
1/31/2005,-0.868115,-0.857824,-0.947757,-0.986296,-1.572234,-0.799742,-0.978562,-1.077044,-2.061492
2/28/2005,-0.774480,-0.805958,-0.916390,-0.941371,-1.570576,-0.958262,-0.951162,-0.968363,-2.021143
3/31/2005,-0.842924,-0.853088,-0.935187,-0.973422,-1.583617,-1.252316,-0.974236,-1.069754,-2.043124
4/29/2005,-0.890692,-0.886012,-0.970707,-1.005875,-1.567827,-0.932974,-0.999472,-0.871614,-2.026562
5/31/2005,-0.826051,-0.836627,-0.928960,-0.974092,-1.543025,-0.815818,-0.958372,-0.743716,-2.078956


In [5]:
cov = pool.cov()
target = [0.7, 0.2, 0.1] # target risk contribution of equity, bond, alternative
equity = 0.8 # equity proportion limitation
wts = risk_parity_weight(cov, target, equity)
wts

iShares Russell 1000 Value ETF         0.222918
Praxis Value Index A                   0.223225
Wells Fargo C&B Large Cap Value A      0.212622
Commerce Value                         0.117451
Metropolitan West Total Return Bd I    0.078259
iShares Core US Aggregate Bond ETF     0.011901
Schwab S&P 500 Index                   0.025214
Vanguard Real Estate ETF               0.036727
iShares Gold Trust                     0.071684
dtype: float64

In [6]:
var = (wts.T).dot(cov).dot(wts)
risk_contribution = wts*(cov.dot(wts))/var
risk_contribution = [sum(risk_contribution[0:3]), sum(risk_contribution[3:5]), sum(risk_contribution[5:9])]
risk_contribution

[0.6999998177750311, 0.20000027564324827, 0.09999990658172059]